In [1]:
import scipy
import numpy as np
from sklearn.neighbors import KernelDensity
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.cluster import estimate_bandwidth
from sklearn.cluster import MeanShift, estimate_bandwidth

import pandas as pd

from scipy import stats
from scipy.stats import beta
from math import sin
from random import randint
from IPython.display import clear_output
import matplotlib.pyplot as plt
import itertools as it

import plotly
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
init_notebook_mode(connected=True)

import collections

def recursively_default_dict():
        return collections.defaultdict(recursively_default_dict)

from matplotlib.collections import BrokenBarHCollection
import re

from structure_tools.Modules_tools import return_fsts

PCA_color_ref= ['darkseagreen','crimson', 'darkorange', 'darkblue', 'darkcyan',
            'darkgoldenrod', 'darkgray', 'darkgrey', 'darkgreen',
            'darkkhaki', 'darkmagenta', 'darkolivegreen', 'darkorange',
            'darkorchid', 'darkred', 'darksalmon', 'darkseagreen',
            'darkslateblue', 'darkslategray', 'darkslategrey',
            'darkturquoise', 'darkviolet', 'deeppink']

## Cluster Seach - Ne estimation.

**Disclaimer**

This notebook is not about imputation. 

**Cluster Search** is an approach used in other notebooks in this directory. It consists in the use of Kernel Density Estimation, in combination with dimensionality reduction (commonly PCA) and clustering (commonly Mean Shift), to identify similar clusters of genetic data across data sets. 

Initially developped in [/Tools_I](https://github.com/SantosJGND/Tools_and_toys) the pipeline is used in this directory for supervised imputation ([section III](https://github.com/SantosJGND/Tools_II/tree/master/Imputation)). 

The approach has also been applied to capture cryptic genetic signatures in Japonica rice (BioRXiv). 

### Effective population size


In this notebook we use the Cluster Search for maximum likelihood estimation of the `Theta` parameter, the scaled mutation rate (4Neµ under neutrality), for targeted clusters of haplotypes. 

The algorithms are developped and presented (succintly..) in another [notebook](https://nbviewer.jupyter.org/github/SantosJGND/Coalescent/blob/master/Coalescence_probability.ipynb). An application example is given [here](https://nbviewer.jupyter.org/github/SantosJGND/Coalescent/blob/master/Faisal_List.ipynb).



## I. vcf analysis
Jupyter notebook for the local analysis of genetic data stored in .vcf format.

Perform analysis of structure across data set, followed by a more detailed study of variation across local genomic windows.

### Input

In [2]:
from structure_tools.vcf_geno_tools import simple_read_vcf
from structure_tools.vcf_geno_tools import read_geno_nanum

vcf_file= 'D:/GitHub/Tools_and_toys/VCF_analysis/Extract/vcf/Extract_Chr8_15000.vcf'

row_info= 6
header_info= 9
phased= False

genotype, summary, Names= read_geno_nanum(vcf_file, row_info= row_info, header_info= header_info,phased= phased)

print('Number of markers: {}'.format(genotype.shape[1]))
print('Number of individuals: {}'.format(genotype.shape[0]))
control_subset= 0

{'fileformat': 'VCFv4.2', 'fileDate': '20190327', 'source': 'PLINKv1.90', 'contig': '<ID8,length28422468>', 'INFO': '<IDPR,Number0,TypeFlag,Description"Provisional reference allele, may not be based on real reference genome">', 'FORMAT': '<IDGT,Number1,TypeString,Description"Genotype">'}
Number of markers: 15000
Number of individuals: 3023


In [3]:
## read passport information

Input_file= 'D:/Rice/Project_external/metadata/orderCore_INFO.txt'

RG_info= pd.read_csv(Input_file,sep= '\t')

RG_info.head()

,Unnamed: 0,ID,NAME,COUNTRY,REGION,sNMF_K3,Jap_K4,K9_cluster,Initial_subpop,genoIndex,code,label
0,0,CX59,"MILAGROSA,_ZAWA_BANDAY",Philippines,As5,4,1,cB_(Bas),aro,296,4,aro
1,1,CX65,DOMSIAH,Iran,As1,4,1,cB_(Bas),aro,301,4,aro
2,2,CX67,BINAM,Iran,As1,4,1,cB_(Bas),aro,303,4,aro
3,3,CX104,SADRI_RICE_1,Iran,As1,4,1,cB_(Bas),aro,338,4,aro
4,4,CX143,KHASAR,Iran,As1,4,1,cB_(Bas),aro,372,4,aro


In [4]:
summary.head()

,CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT
0,8,14473,242044001,C,T,.,.,PR,GT
1,8,15216,242044744,T,C,.,.,PR,GT
2,8,18535,242048063,G,A,.,.,PR,GT
3,8,19970,242049498,C,T,.,.,PR,GT
4,8,26680,242056208,C,A,.,.,PR,GT


In [5]:
## Process Names vcf names.
## Instance specific processing due to ID copy in VCF file.
Names_vcf= list(Names)

for x in range(len(Names_vcf)):
    ind= Names_vcf[x]
    newid= ind.split('_')
    
    if len(newid) > 2:
        newid= '_'.join(newid[:2])
    else:
        newid= newid[0]
    
    Names_vcf[x]= newid


In [6]:
### subset core
coreID_file= 'D:/Rice/Project_external/metadata/Order_core.txt'
with open(coreID_file,'r') as fp:
    coreIDs= fp.readlines()

coreIDs= [x.strip() for x in coreIDs]

core_idx= [x for x in coreIDs if x in Names_vcf]
core_idx= [Names_vcf.index(x) for x in core_idx]

ID_pop= {
    RG_info['ID'][x]: RG_info['Initial_subpop'][x] for x in range(RG_info.shape[0])
}

core_names= [Names_vcf[x] for x in core_idx]
core_pop= [ID_pop[x] for x in core_names]


In [7]:

if control_subset == 0:
    genotype= genotype[core_idx,:]
    control_subset+= 1
    
    ###
    gen_sum= np.sum(genotype,axis= 0)
    gen_sum= gen_sum != 0

    genotype= genotype[:,gen_sum]
    summary= summary.iloc[gen_sum]
    summary= summary.reset_index()


In [8]:

sub_sel= ['subtrop','trop','temp','aro']

if sub_sel: 
    sub_idx= [x for x in range(len(core_pop)) if core_pop[x] in sub_sel]
    genotype= genotype[sub_idx]
    core_pop= [core_pop[x] for x in sub_idx]
    core_names= [core_names[x] for x in sub_idx]



## II. Global variation

Perform PCA across data set.

Perform Mean shift clustering to attempt to extract genetically coherent groups of accessions.

These will later be used for supervised analysis.

In [9]:
## Perform PCA
n_comp= 4
pca = PCA(n_components=n_comp, whiten=False,svd_solver='randomized')

feats= pca.fit_transform(genotype)

In [10]:
from structure_tools.Tutorial_subplots import plot_global_pca
## perform MeanShift clustering.
bandwidth = estimate_bandwidth(feats, quantile=0.2)

ms = MeanShift(bandwidth=bandwidth, bin_seeding=False, cluster_all=True, min_bin_freq=15)
ms.fit(feats)
labels1 = ms.labels_
label_select = {y:[x for x in range(len(labels1)) if labels1[x] == y] for y in sorted(list(set(labels1)))}

###
label_pops= {z: [core_pop[x] for x in g] for z,g in label_select.items()}
label_connect= {z: g[np.random.randint(0,len(g))] for z,g in label_pops.items()}
colordict= {z: PCA_color_ref[z] for z in label_pops.keys()}




In [11]:
###
plot_global_pca(feats,label_select,colordict,labels= core_pop,title= 'global_pca',height= 500,width= 950)

c:\users\floyd\appdata\local\programs\python\python37-32\lib\site-packages\plotly\tools.py:465: DeprecationWarning:

plotly.tools.make_subplots is deprecated, please use plotly.subplots.make_subplots instead



### AMOVA at local windows

Between population variance at local windows. 

#### I. Local windows

`Nwindows`: number of windows

`Wsizes`: size of each window (in feature number).


In [12]:
SequenceStore= {}

Nwindows= 600
Wsizes= 150
chrom= 1

wst= np.random.randint(0,genotype.shape[1] - Wsizes,size= Nwindows)
wst= np.linspace(0,genotype.shape[1] - Wsizes,Nwindows,dtype= int)
SequenceStore= {
    chrom: {summary.POS[st]: genotype[:,st:(st+Wsizes-1)] for st in wst}
}

Out= {chrom: {summary.POS[st]: summary.POS[st+ Wsizes - 1]for st in wst}}



#### AMOVA

Calculating local AMOVA. Chose which groups to calculate AMOVA between at each window using the labels in the first plot in `ref_gps` (above).

In [13]:
ref_gps= [0,1,3,4]


In [14]:
## AMOVA parameters.
supervised= True

Bandwidth_split= 30 # grid split for kde 
KDE_comps= 4 # PCA components to retain
clsize= 10 # minimum cluster size to retain during ms clustering.
amova= True # whether to calculate amova.


In [15]:
from structure_tools.StructE_tools import findPhiPT, Structure_profiles, Distance_profiles

from structure_tools.AMOVA_func import amova_cofactor, AMOVA_FM42
from structure_tools.mstutorial_tools import Windows_KDE_amova

### Perform Distance and association analysis on the data sets generated

refs_lib= {v:g for v,g in label_select.items() if v in ref_gps}
admx_lib= {v:g for v,g in label_select.items() if v not in ref_gps}
admx_lib.update(refs_lib)
amova_toggle= True
char_avoid= [9,1]
ind_thresh= 0.01

Results, Construct, PC_var= Windows_KDE_amova(SequenceStore,
                                              admx_lib,
                                              refs_lib,
                                              supervised= supervised,
                                              amova= amova_toggle,
                                              ncomps= KDE_comps,
                                              clsize= clsize,
                                              Bandwidth_split= Bandwidth_split,
                                              char_avoid= char_avoid,
                                              ind_thresh= ind_thresh)

chr 1, where: 28061562, supervised: True, n clusters: 4
old: ; jaccard: 0.017398338709608926; PCA euc: 0.05709604065602962; nHam: 0.019369818400213814


In [16]:
AMOVA_stats= [[[Chr,wind,*Results[Chr][wind]] for wind in Results[Chr].keys()] for Chr in Results.keys()]
AMOVA_stats= np.array([y for y in it.chain(*AMOVA_stats)])

Names= ['updt jaccard','updt euc','updt hamming']

fig_data= [go.Scatter(
    x= AMOVA_stats[:,1],
    y= AMOVA_stats[:,x],
    mode= 'markers',
    name= Names[x - 3]
) for x in range(3,6)]

layout = go.Layout(
    title= 'Stats',
    yaxis=dict(
        title='AMOVA'),
    xaxis=dict(
        title='Windows')
)

fig= go.Figure(data=fig_data, layout=layout)
iplot(fig)

## III. Cluster Focus

In this section we identify clusters at local windows. For each cluster we will extract the p-value of every accession given that cluster. This p-value is calculated using the cluster kernel density estimation in PCA feature space (_MS profile_).

Cluster profiles are captured as normalized individual cdfs under specific cluster kernel density estimates in feature space.

The identification of clusters in feature space can be done using any of the methods available in the sklearn package. The method and parameters are defined in the `CL_store` dictionary below. 

First however we define which accessions to use in identifying clusters and extracting cluster profiles. 



#### i. Accession focus

Once again we can chose to use only a subset of accessions with which to identify clusters. To select these accessions use the labels.


In [17]:

select_refs= [0,1,3,4]
#select_refs= ['temp','subtrop','aro']#list(set(labels1))
#labels1= [label_connect[x] for x in labels1] 

label_vector= [[len(select_refs),labels1[x]][int(labels1[x] in select_refs)] for x in range(genotype.shape[0])]

Whose= list(range(genotype.shape[0]))


In [18]:
Names=list(core_names)
Fam= {
    Names[x]:x for x in range(len(Names))
}

Fam.update({
    x:Names[x] for x in range(len(Names))
})

###
Dr_dim= 4

###
focus_subset= False
Geneo= admx_lib
Focus_group= list(select_refs)

Focus= [core_names[x] for x in range(len(Names)) if core_pop[x] in Focus_group]

###
Dr_var= 'all' #'all'
target_var= list(ref_gps)

##

#### ii. Local window custer profiles

In [19]:
from sklearn.cluster import DBSCAN
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import KMeans

Method= 'MeanShift'

Cl_store= {
    'MeanShift':{
        'Clusterfunc': MeanShift,
        'cluster_kwargs': {
            'bin_seeding': True,
            'cluster_all': False,
            'min_bin_freq': 5
        }
    },
    'DBscan':{
        'Clusterfunc': DBSCAN,
        'cluster_kwargs': {
            'min_samples': 15
        }
    },
    'Ward':{
        'Clusterfunc': AgglomerativeClustering,
        'cluster_kwargs': {
            'linkage': 'ward',
            'n_clusters': 4
        }
    },
    'Kmeans':{
        'Clusterfunc': KMeans,
        'cluster_kwargs': {
            'random_state': 0,
            'n_clusters': 3
        }
    }
}

In [20]:
from structure_tools.mstutorial_tools import MAC_process

preProc_Clover, Cameo, Coordinates, COMPS, X_se, label_comps, Subset, labels_comp= MAC_process(Construct,
                                                                             Out,
                                                                             Cl_store,
                                                                             refs_lib,
                                                                             Fam,
                                                                             Names= Names,
                                                                             target_var= target_var,
                                                                             Dr_var= Dr_var,
                                                                             focus_subset= focus_subset,
                                                                             Focus= Focus,
                                                                             Dr_dim= Dr_dim,
                                                                             Method= Method)


Clover shape:  (2058, 385)
Clover shape:  (2058, 385)
focusing Dr on all


#### iii. cluster visualisation.

The function `MAC_process` performs dimensionality reduction on the _MS profiles_ extracted across local windows. _MS profiles_ are clustered (PCA + Mean Shift). 

Feature-wise, MS profiles contain the normalized probability of individual accessions towards window-level genetic clusters. We average MS_profiles by group. 

Below, the principal coordinates of Dr on the entire data sets are used to convey the ID of _MS profile_ features.

First the labels from the full set clustering are replotted.


In [21]:
from plotly import tools
from structure_tools.mstutorial_tools import KDE_pca

KDE_pca(feats= feats,Cameo= Cameo,label_vector= labels1,Subset= Subset, 
        Col_vec= colordict)

['Global', 'Global', 'cluster 1', 'cluster 1', 'cluster 2', 'cluster 2', 'cluster 3', 'cluster 3', 'cluster 4', 'cluster 4', 'cluster 5', 'cluster 5', 'cluster 6', 'cluster 6', 'cluster 7', 'cluster 7', 'cluster 8', 'cluster 8', 'cluster 9', 'cluster 9', 'cluster 10', 'cluster 10']


c:\users\floyd\appdata\local\programs\python\python37-32\lib\site-packages\plotly\tools.py:465: DeprecationWarning:

plotly.tools.make_subplots is deprecated, please use plotly.subplots.make_subplots instead



## IV. Reconstruct target tree. 

We find that _MS profile_ groups correspond to specific clusters that show up in global Dr.

We also find that other _MS profile_ groups encompass more than one group of observations.

This is indicative of internal structure: At some local windows observations from two or more groups are found to cluster together. This can have several interpretations depending on your data. We are interested in capturing this data. 

We are going to focus on the clusters observed in the global Dr.

We are going to calculate distances between these clusters at windows where all three can be identified. We then compare these local distances to the global estimates calculating the whole data set.

The list `cluster_include` below determines which MS_profiles we chose to target. 

**Method** 

We will use the _MS profile_ groups displayed below as trainning sets. _MS profiles_ are stored in array `preProc_Clover` groups indexed in `label_comps`. This information is used to classify clusters identified at local windows. 

**cluster classification**

The array `preProc_Clover` is reduced and the kd of target (cluster_include) MS profile groups is estimated in feature space. 

Maximum likelihood classificaiton of Local _MS profiles_  is performed, allowing for outliers (max(L) < threshold). Outliers are discarded.

#### i. Select clusters

In [22]:
cluster_include= [1,2,3,5]

comp_label_keep= {z:g for z,g in label_comps.items() if z in cluster_include}
cameo_label= {z: [x for x in range(Cameo.shape[0]) if Cameo[x,z] > .05] for z in range(Cameo.shape[1])}

#### ii. generate windows

random or linear.

In [23]:
Nwindows= 500 # Number of windows
Wsizes= 150 # sizes in number of features
chrom= 1

wst= np.random.randint(0,genotype.shape[1] - Wsizes,size= Nwindows)
wst= np.linspace(0,genotype.shape[1] - Wsizes,Nwindows,dtype= int)
SequenceStore= {
    chrom: {summary.POS[st]: genotype[:,st:(st+Wsizes-1)] for st in wst}
}

Out= {chrom: {summary.POS[st]: summary.POS[st+ Wsizes - 1]for st in wst}}


#### iii. target clusters, calculate distances

In [24]:
from structure_tools.MS_target_tools import (
    MS_get_norm, kde_gen_dict, 
    gen_class, clustClass, D1_kdegen,
    plot_distances, target_MSP
)


In [25]:
from IPython.display import clear_output
from sklearn.metrics import pairwise_distances

pca_qtl= 0.1
ncomps= 4
clsize= 15
Bandwidth_split= 20
out_code= -1
metric= 'euclidean'
lb= 1e-2
cl_samp= 50

char_avoid= [9,1]
ind_thresh= 1 / (Wsizes+1)

Geneo= admx_lib
Geneo_order= list(Geneo.keys())
ref_order= list(refs_lib.keys())

Whose= list(range(sum([len(x) for x in Geneo.values()])))
Sup_labels= list(np.repeat(Geneo_order,[len(Geneo[x]) for x in Geneo_order]))

### Define parameters and libraries of analyses.

Results = {x:recursively_default_dict() for x in SequenceStore.keys()}

###
###

dists_dict= target_MSP(SequenceStore[chrom],preProc_Clover, comp_label_keep, refs_lib, Whose,
               ncomps= ncomps, clsize= clsize, Bandwidth_split= Bandwidth_split, out_code= out_code,
               metric= metric, cl_samp= cl_samp)


#### iv. plot

We gathered the distances between target clusters across windows. Below we plot, for each target cluster, the distribution of distance to every other target cluster.

In [26]:

range_dists= np.linspace(0,12,100)
range_dists= range_dists.reshape(-1,1)


dist_gens= {}

for gp in sorted(dists_dict.keys()):
    gp_sub_gens= plot_distances(dists_dict,gp,range_dists,height= 500,width= 900)
    dist_gens[gp]= gp_sub_gens


['cl: 2', 'cl: 3', 'cl: 5']
0
1
2


['cl: 1', 'cl: 3', 'cl: 5']
0
1
2


['cl: 1', 'cl: 2', 'cl: 5']
0
1
2


['cl: 1', 'cl: 2', 'cl: 3']
0
1
2


## V. Imputing a specific cluster

We will use the distance profiles to provide the most likely positions of a missing cluster at a local window. 

We select one cluster to be 'absent'. 

We use the tools developed above to identify clusters at the local window that correspond to our targets.

We use the identified clusters as reference points to estimate the most likely position the _absent_ cluster in feature space.

### i. generate local window.


### iii. Cluster target.

1. Identify clusters at local window. 

In [95]:

from structure_tools.MS_target_tools import (
    clust_samp, comb_score
)

print('full data set shape: {}'.format(genotype.shape))

wind_sizes= 150
ncomps= 4
chrom= 1
dimN= 2
ncomps= dimN

metric= 'euclidean'

nan_n= 1

##
clust_nd= 3


###
###
clov_pca= PCA(n_components=ncomps, whiten=False,svd_solver='randomized').fit(preProc_Clover)
data_clov= clov_pca.transform(preProc_Clover)

ref_gens, ref_stats= kde_gen_dict(data_clov,comp_label_keep,
                    Bandwidth_split= Bandwidth_split)

d= 0
while d == 0:
    xnan= np.random.randint((wind_sizes)/2+1,genotype.shape[1] - (wind_sizes)/2,size= nan_n)
    ynan= np.random.randint(0,genotype.shape[0],size= nan_n)

    nan_coords= [ynan,xnan]
    nan_coords= np.array(nan_coords).T

    nan_idx= 0

    nan_obs= nan_coords[nan_idx]
    #nan_obs= [43,16000]
    nan_acc= nan_obs[0]
    nan_pos= nan_obs[1]
    
    local_l= genotype[:,(nan_pos-int(wind_sizes/2)):(nan_pos+int(wind_sizes/2))]

    ###
    ###

    lb= 1e-3
    Bandwidth_split= 20
    pca_qtl= .1


    lclust_samp, lclust_gens, feat_seq= clust_samp(local_l, refs_lib,clov_pca, ref_gens, ref_stats,
                  ncomps= ncomps,clsize= clsize,Bandwidth_split= Bandwidth_split,
                    pca_qtl= pca_qtl, return_acc= True)
    
    if len(lclust_samp) >= clust_nd:
        d+= 1


print('clusters identified a local window: ')
print(lclust_samp.keys())
found_keys= list(lclust_samp.keys())

full data set shape: (385, 8770)
clusters identified a local window: 
dict_keys([5, 1, 2])


In [96]:

#local_l= SequenceStore[1]['757862']
coords= {z:[x for x in range(len(label_vector)) if label_vector[x] == z] for z in list(set(label_vector))}

pca2 = PCA(n_components=ncomps, whiten=False,svd_solver='randomized')
featl= pca2.fit_transform(local_l)

figwl= [go.Scatter(
    x= featl[coords[i],0],
    y= featl[coords[i],1],
    mode= 'markers',
    name= str(i)
) for i in coords.keys()]


cols_avail= [
    'red','blue','green'
]
 

cols_cl= {found_keys[x]:cols_avail[x] for x in range(len(cols_avail))}

figwl.extend([go.Scatter(
    x= [np.mean(g,axis= 0)[0]],
    y= [np.mean(g,axis= 0)[1]],
    mode= 'markers',
    marker=dict(
    color='rgba(135, 206, 250, 0)',
    size=50,
    opacity= 1,
    line=dict(
        color=cols_cl[v],
        width=5
    )
),
    name= str(v)
) for v,g in lclust_samp.items()])


layout= go.Layout()

Figure_wl= go.Figure(data= figwl, layout= layout)

iplot(Figure_wl)

# VI. Cluster theta

Select one of the clusters identified at the local window (previous section).


In [106]:

grp_idx= 1

### Pre-processing

Some filters to ensure we are dealing with haplotypes.

Rice data is very homozygous. We will play with it as if these were haplotypes. Cluster Search functions ignores and excludes haplotypes with missing or heterozygous calls. 



In [107]:

hap_window= local_l>0
hap_window= np.array(hap_window,dtype= int)
###

data_window= hap_window[feat_seq[grp_idx]]

### assume all individuals are unphased homozygous genotypes. transform to haplotypes.

sub_sel_method= 'ms'
Ng= 2
Anc_pop_1= True
mrca_pop= 0


n_comp= 4

In [108]:
data_window.shape

(31, 150)

### Prepare for coalescence inference

Coalescence approaches get very expensive very quickly. We limit the number of haplotypes in this data to a manageable number.

In [109]:
# max number of samples from this pop
max_sample= 12

### rand sample

if data_window.shape[0] < max_sample:
    vec_samp= list(range(data_window.shape[0]))
else:
    vec_samp= np.random.choice(list(range(data_window.shape[0])),max_sample,replace= False)
        

#######
vec_samp= sorted(vec_samp)
data_w= data_window[vec_samp,:]


if Anc_pop_1:
    
    n_comp= 4
    pca = PCA(n_components=n_comp, whiten=False,svd_solver='randomized')
    featsw= pca.fit_transform(data_window)
    feats_mean= np.mean(featsw,axis= 0)
    feat_cent= pca.inverse_transform(feats_mean)
    mrca_hap= np.array(feat_cent,dtype= int)
    
    data_w= [[int(z[x] != mrca_hap[x]) for x in range(len(mrca_hap))] for z in data_w]
        
    data_w= np.array(data_w)


### remove monomorph:
sum_gen= np.sum(data_w,axis= 0)
sum_gen= sum_gen > 0
data_w= data_w[:,sum_gen]


## Coalescence algorimths.

### I. Infinite sites.



In [115]:
from structure_tools.Coal_index import process_array, get_config
### example from figure 2.10.


Dict_mat, point_up, config_dataw= process_array(data_w, return_config= True)


In [116]:
from structure_tools.Coal_index import Inf_sites

point_up= recursively_default_dict()
point_dn= recursively_default_dict()

root_lib, point_up = Inf_sites(Dict_mat,point_up,layer_range= 36,sub_sample= 0,poppit= False,print_layer= False)

time elapsed: 0.018 s


In [117]:
from structure_tools.Coal_probab import Ascent_return, tree_ascent
from structure_tools.Coalesce_plots import plot_rec_InfSites

func_names= ['tree_construct']
funcs= [
        Ascent_return      # runUp_balance # tree_construct
       ]

range_theta= np.linspace(0.01,15,100)

plot_rec_InfSites(point_up,root_lib,funcs,func_names,range_theta,height= 500)

### Infinite Alleles

In [118]:
import math

def Ewens_exact(config_data,theta):
    
    n_samp= sum([(x + 1) * config_data[x] for x in range(len(config_data))])
    
    ThetaN= [theta + x for x in range(len(config_data))]
    ThetaN0= 1
    for y in ThetaN:
        ThetaN0 = ThetaN0 * y
    
    factor_front= math.factorial(len(config_data)) / ThetaN0
    
    classes= 1
    
    for j in range(len(config_data)):
        comb= theta / (j+1)
        comb= comb**config_data[j]
        
        prod= comb * (1 / math.factorial(config_data[j]))
        
        classes= classes * prod
    
    return factor_front * classes

####


Browse= Ewens_exact(config_dataw, range_theta)


In [119]:
trace1= [go.Scatter(
    y= Browse,
    x= range_theta,
    mode= 'markers'
)]


layout= go.Layout(
    xaxis= dict(title= 'Theta'),
    yaxis= dict(title= 'P')
)

Figure= go.Figure(data= trace1, layout= layout)
iplot(Figure)

## Conclusion

At least on this group, it is apparent that there are differences between the maximum likelihood estimates using infinite alleles and infinite sites approaches. This is to be expected. 

While the infinite sites approaches can be taken to be closer to the _truth_, it also becomes very expensive with the complexity of the ancestral graph, the number of possible ancestral states. Unless there is very little diversity, this complexity increases rapidly with the number of samples and features.

Rapidly enough that Ewen's approach might become a reasonable compromise..

### Optimizing Theta.

Weighed likelihood sampling for more accurate measures of `theta`. 

In [53]:

def theta_opt(runUp_use,tmin= 0.01,tmax= 15,nr= 20):
    '''
    '''
    range_theta= np.linspace(0.01,15,nr)
    Inf_sites_est= []
    coords= []


    d= 0

    while d == 0:
        
        current_layer= []
        there= []
        for x in range_theta:

            ## run up the tree.
            Browse= runUp_use(point_up,root_lib,layer=0,start=0,Theta= x,prob_vec= [])
            probe_rec= sum(Browse)

            current_layer.append(probe_rec)
            there.append(x)

        if len(Inf_sites_est):
            if np.mean(current_layer) < (np.mean(Inf_sites_est)+np.std(Inf_sites_est)) or np.std(current_layer) == 0:
                Inf_sites_est.extend(current_layer)
                coords.extend(there)
                d += 1

        max_vals= [x for x in range(len(current_layer)) if current_layer[x] > np.mean(current_layer)]
        nrange= [there[x] for x in max_vals]
        range_theta= np.linspace(min(nrange),max(nrange),nr)
        Inf_sites_est.extend(current_layer)
        coords.extend(there)
    
    datum= [coords,Inf_sites_est]
    datum= np.array(datum).T
    
    return datum


nr= 20
tmin= 0.01
tmax= 15
runUp_use= Ascent_return

theta_array= theta_opt(runUp_use,tmin= 0.01,tmax= 15,nr= 20)

theta_select= np.argmax(theta_array,axis= 0)
theta_select= theta_array[theta_select[1],0]
theta_select


0.7989473684210526